# RNN - Recurrent Neural Network

### Dependencias

In [ ]:
import numpy as np
import tensorflow
np.random.seed(5)

from tensorflow.keras.layers import Input, Dense, SimpleRNN
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
#from keras import backend as K

### Lectura del conjunto de datos

In [ ]:
# 1. LECTURA DEL SET DE DATOS
# ===========================================================
# Abre el archivo nombres_dinosaurios.txt en modo de lectura y se le asgina a la variable llamada nombre
nombres = open('nombres_dinosaurios.txt','r').read()
# Se ponen todas las minúsculas los nombres de los dinosaurios
nombres = nombres.lower()

# Crear  listado de caracteres que no se repiten
alfabeto = list(set(nombres))
''' 
	- tam_alfabeto :  Se obtiene la cantidad de letras (Longitud de la lista alfabeto)
	- tam_datos :     Se obtiene la cantidad de caracteres de cada dinosausrio (Incluye repetidos)
'''
tam_datos, tam_alfabeto = len(nombres), len(alfabeto)
# Imprime el número de caracteres y el número de 
print("En total hay %d caracteres, y el diccionario tiene un tamaño de %d caracteres." % (tam_datos, tam_alfabeto))

# Conversión de caracteres a índices y viceversa
'''
Ordena los caracteres en orden alfábetico y a cada uno le asigna un número.
Esto incluye saltos de linea "\n".
	- En cara_a_ind se le asigna un número a cada elemento de alfabeto
	- En ind_a_cara se le asigna un elemento de alfabeto a un número.

'''
car_a_ind = { car:ind for ind,car in enumerate(sorted(alfabeto))}
ind_a_car = { ind:car for ind,car in enumerate(sorted(alfabeto))}

#Se imprimen los resultados
print(car_a_ind)
print(ind_a_car)


En total hay 19909 caracteres, y el diccionario tiene un tamaño de 27 caracteres.
{'\n': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
{0: '\n', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


### Construcción del modelo

In [ ]:
# 2. MODELO
# ===========================================================
n_a = 25    # Número de unidades en la capa oculta , numero de neuronas 
#Entradas de nombres de dinosaurios con tamaño variable 
entrada  = Input(shape=(None,tam_alfabeto)) #Se crea una tupla con las dimensiones (0,0,27(Alfabeto)
a0 = Input(shape=(n_a,))    #Crea una tupla con las dimensiones (0,0,25)

#Generar el nuevo estado oculto usando Tanh ,return_state : a la salida retorna el nuevo estado oculto
celda_recurrente = SimpleRNN(n_a, activation='tanh', return_state = True) #Crear el modelo de redes recurrente 
#Generar la salida o prediccion ,27 neuronas de salida (alfabeta )
capa_salida = Dense(tam_alfabeto, activation='softmax') #Crear la capa densa , con la funcion de activacion softmax

salida = [] #Crear una lista para almacenar la salida  de la prediccion 
hs, _ = celda_recurrente(entrada, initial_state=a0) #Fijar el modelo con un tamaño de 25 como entrada y 25 como salida
salida.append(capa_salida(hs))  #Agregar la capa con el modelo al arreglo de salida
#Agrupar los dos input, el simpleRNN y la capa densa en la variable modelo
#2 entradas  el caracter actual y el estado oculto anterior
modelo = Model([entrada,a0],salida)
modelo.summary()

#OPtimizador 
opt = SGD(lr=0.0005) #Se define el gradiente decendente
modelo.compile(optimizer=opt, loss='categorical_crossentropy') #Agregar el gradiente al modelo

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, None, 27)]   0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 25)]         0           []                               
                                                                                                  
 simple_rnn (SimpleRNN)         [(None, 25),         1325        ['input_1[0][0]',                
                                 (None, 25)]                      'input_2[0][0]']                
                                                                                                  
 dense (Dense)                  (None, 27)           702         ['simple_rnn[0][0]']         

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


### Generación del conjunto de entrenamiento

In [ ]:
# 3. EJEMPLOS DE ENTRENAMIENTO
# ===========================================================
# Whit : Determinar la configuración local que tendrá un bloque de código, lo que se conoce como "contexto".
# Crear lista con ejemplos de entrenamiento y mezclarla aleatoriamente
with open("nombres_dinosaurios.txt") as f:          # como primer paso volvemos cada linea del archivo como un elemento del objeto lista
    ejemplos = f.readlines()                        # despues se transforma a minusculas y se les agregan espacios 
ejemplos = [x.lower().strip() for x in ejemplos]    # por ultimos se mezcla su contenido sin alterarlo.
np.random.shuffle(ejemplos)

# Crear ejemplos de entrenamiento usando un generador
def train_generator():
    while True:
        # Tomar un ejemplo aleatorio
        ejemplo = ejemplos[np.random.randint(0,len(ejemplos))]

        # Convertir el ejemplo a representación numérica
        X = [None] + [car_a_ind[c] for c in ejemplo]

        # Crear "Y", resultado de desplazar "X" un caracter a la derecha
        Y = X[1:] + [car_a_ind['\n']]

        # Representar "X" y "Y" en formato one-hot            # crea una columna binaria para cada categoría, en una matriz numerica.
        x = np.zeros((len(X),1,tam_alfabeto))
        onehot = to_categorical(X[1:],tam_alfabeto).reshape(len(X)-1,1,tam_alfabeto)
        x[1:,:,:] = onehot
        y = to_categorical(Y,tam_alfabeto).reshape(len(X),tam_alfabeto)

        # Activación inicial (matriz de ceros) , se actualiza a medida que se le presentan los caracteres al modelo 
        a = np.zeros((len(X), n_a))

        yield [x, a], y  #mantenemos los valores en el espacio de memoria 

### Entrenamiento de la red

In [ ]:
# 4. ENTRENAMIENTO
# ===========================================================
BATCH_SIZE = 80			# Número de ejemplos de entrenamiento a usar en cada iteración
NITS = 10000			# Número de iteraciones
#Ciclo para iniciar las iteraciones, la cuales son 10000
for j in range(NITS):
    '''
    Se inicia el entrenamiento con el modelo definidio en la sección dos
    Por medio de la función train_generator definidia en la sección 3 se crean ejemplos de entrenamiento usando un generador+
    steps_per_epoch  es un parámetro que representa el número de iteraciones por lotes antes de que una época
      de entrenamiento se considere terminada.
    verbose=0 hace referencia que no va a mostrar nada como salida
    epochs=1 hace referencia al número de iteraciones sobre los datos proporcionados
	
    '''
    historia = modelo.fit_generator(train_generator(), steps_per_epoch=BATCH_SIZE, epochs=1, verbose=0)

    # Imprimir evolución del entrenamiento cada 1000 iteraciones
    # Támbién muestra el historial de pérdidas
    if j%1000 == 0:
        print('\nIteración: %d, Error: %f' % (j, historia.history['loss'][0]) + '\n')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  import sys



Iteración: 0, Error: 3.340127


Iteración: 1000, Error: 2.398685


Iteración: 2000, Error: 2.253265


Iteración: 3000, Error: 2.175043


Iteración: 4000, Error: 2.186652


Iteración: 5000, Error: 2.171052


Iteración: 6000, Error: 2.143520


Iteración: 7000, Error: 2.237665


Iteración: 8000, Error: 2.203322


Iteración: 9000, Error: 2.175839



### Generación de nombres usando el modelo entrenado

In [ ]:
# 5. GENERACIÓN DE NOMBRES USANDO EL MODELO ENTRENADO
# ===========================================================
def generar_nombre(modelo,car_a_num,tam_alfabeto,n_a):
    # Inicializar x y a con ceros
    x = np.zeros((1,1,tam_alfabeto,))
    a = np.zeros((1, n_a))

    # Nombre generado y caracter de fin de linea
    nombre_generado = ''
    fin_linea = '\n'
    car = -1

    # Iterar sobre el modelo y generar predicción hasta tanto no se alcance
    # "fin_linea" o el nombre generado llegue a los 50 caracteres
    contador = 0
    while (car != fin_linea and contador != 50):
          # Generar predicción usando la celda RNN
          #Se lleva a la capa softmax para asi generar la prediccion 
          a, _ = celda_recurrente(K.constant(x), initial_state=K.constant(a))
          #La predccion tendra un vector de 27 elementos que representa una distribucion de probabilidad 
          y = capa_salida(a)
          # Evalua el palor de la variable y
          prediccion = K.eval(y)

          # Escoger aleatoriamente un elemento de la predicción (el elemento con
          # con probabilidad más alta tendrá más opciones de ser seleccionado)
          ix = np.random.choice(list(range(tam_alfabeto)),p=prediccion.ravel())

          # Convertir el elemento seleccionado a caracter y añadirlo al nombre generado
          car = ind_a_car[ix]
          nombre_generado += car

          #se actualiza las entradas , se genera la realimentacion , la prediccion generada se volvera la entrada de la siguiente 

          # Crear x_(t+1) = y_t, y a_t = a_(t-1)
          x = to_categorical(ix,tam_alfabeto).reshape(1,1,tam_alfabeto)
          # Evalua el palor de la variable y
          a = K.eval(a)

          # Actualizar contador y continuar
          contador += 1

          # Agregar fin de línea al nombre generado en caso de tener más de 50 caracteres
          if (contador == 50):
            nombre_generado += '\n'

    #Imprime el nombre que se generó
    print(nombre_generado)

# Generar 100 ejemplos de nombres generados por el modelo ya entrenado
for i in range(100):
    generar_nombre(modelo,car_a_ind,tam_alfabeto,n_a)


kushuruhocosana

patuapsbalodoptrlasaurusaxadusaposaurzuseplrusauar

kapostasauruchsaurosapsaususaphropezheruryptosaury

aposasasausauralurzworgicatrusatarunomogdornomauro

nsalelosex

ramaraushus

alacazerthnn

ocosalosaposatostivlogruryosatosasasauralarausaurg

rsanhonnklisaurusausaurausaushururachoichxtoclthao

csanisauraushtachpasurosactsourausauromelleles

eltels

zx

hima

daurhuaroerhoseirmbryusushuraisadosatergatronagosa

llbrainymergler

eragahosanixlasucosashorolasmictata

ltiuroilamertryptrhimauraushys

huos

megg

gigaphsaytrdoercechmosasasaurees

cosauros

gr

llltrocaprnonalrakorothipksus

ng

dolocephstispass

amorrangodopta

loctaos

aces

traururusys

jura

maururonsavosailolisprocosts

sis

tr

athos

lalosaorolaltrusapszrusa

tacrcasasasaukanatecosagosanhira

dorosaggyngyorhaosaborusancicushanagsosauraurasach

dosps

catiretosaiaslirageosatodosaveruspsa

aroosapososaptitathoitylbryonarechasahuronmnelllta

psausaurusasausurusasa

tinoprsusausberanosachosaus

hivaliazh